In [1]:
%cd /kaggle/working

import os

from hydra import compose, initialize
from omegaconf import OmegaConf

with initialize(version_base=None, config_path="../experiments/032_feature_channel"):
    cfg = compose(
        config_name="config.yaml", overrides=["debug=True"], return_hydra_config=True
    )

/kaggle/working


In [2]:
import warnings
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import polars as pl
import seaborn as sns

warnings.filterwarnings("ignore", "is_categorical_dtype")
warnings.filterwarnings("ignore", "use_inf_as_na")

In [11]:
df = pl.read_parquet("input/train.parquet", n_rows=1)

In [12]:
import xarray as xr

grid_path = "/kaggle/working/misc/grid_info/ClimSim_low-res_grid-info.nc"
grid_info = xr.open_dataset(grid_path)
hyai = grid_info["hyai"].to_numpy()
hybi = grid_info["hybi"].to_numpy()

In [13]:
p0 = 1e5
ps = df["state_ps"].to_numpy()
pressures_array = hyai * p0 + hybi[None, :] * ps[:, None]
pressures_array = np.diff(pressures_array, n=1)

In [14]:
pressures_array

array([[   4.49334817,    8.05875326,   14.30424243,   24.96122521,
          42.45788005,   69.74403971,  109.74026562,  164.59057361,
         235.29041485,  322.19294405,  426.05428239,  548.41345205,
         690.3805663 ,  850.32906287, 1022.31781732, 1197.09251962,
        1365.82669363, 1524.25914584, 1673.82773233, 1818.40857522,
        1958.89747468, 2089.80538655, 2202.61800568, 2313.74224914,
        2363.9771264 , 2390.03109911, 2407.30908291, 2431.73650427,
        2473.40605267, 2534.16905217, 2609.58752016, 2693.01569028,
        2778.85349906, 2863.12716733, 2941.54281821, 3006.87949088,
        3048.13199344, 3052.86196189, 3012.19256048, 2925.85859014,
        2804.03951687, 2664.18047104, 2524.02710921, 2394.62790479,
        2277.17661965, 2165.18830955, 2050.36396987, 1928.62235891,
        1803.08740666, 1682.86437945, 1578.86385644, 1499.29461107,
        1447.09304302, 1420.01895916, 1412.61887973, 1418.626848  ,
        1432.6977876 , 1451.11266244, 1471.68984

In [15]:
pre = hyai * p0 + hybi[None, :] * ps[:, None]

In [16]:
import math

# 定数の定義
T_freeze = 273.15  # 氷点 [K]
ep = 0.622  # 乾燥空気と水蒸気のガス定数の比
pressure = 100000.0  # 全圧 [Pa]


def sat_vapor_press_liq(T):
    """
    液体水に対する飽和蒸気圧を計算する関数
    Bolton (1980) の近似式を使用
    """
    return 611.2 * math.exp((17.67 * (T - T_freeze)) / (T - 29.65))


def sat_vapor_press_ice(T):
    """
    氷に対する飽和蒸気圧を計算する関数
    """
    return 100 * math.exp(23.33086 - (6111.72784 / T) + (0.15215 * math.log(T)))

In [17]:
df.head()

sample_id,state_t_0,state_t_1,state_t_2,state_t_3,state_t_4,state_t_5,state_t_6,state_t_7,state_t_8,state_t_9,state_t_10,state_t_11,state_t_12,state_t_13,state_t_14,state_t_15,state_t_16,state_t_17,state_t_18,state_t_19,state_t_20,state_t_21,state_t_22,state_t_23,state_t_24,state_t_25,state_t_26,state_t_27,state_t_28,state_t_29,state_t_30,state_t_31,state_t_32,state_t_33,state_t_34,state_t_35,…,ptend_v_31,ptend_v_32,ptend_v_33,ptend_v_34,ptend_v_35,ptend_v_36,ptend_v_37,ptend_v_38,ptend_v_39,ptend_v_40,ptend_v_41,ptend_v_42,ptend_v_43,ptend_v_44,ptend_v_45,ptend_v_46,ptend_v_47,ptend_v_48,ptend_v_49,ptend_v_50,ptend_v_51,ptend_v_52,ptend_v_53,ptend_v_54,ptend_v_55,ptend_v_56,ptend_v_57,ptend_v_58,ptend_v_59,cam_out_NETSW,cam_out_FLWDS,cam_out_PRECSC,cam_out_PRECC,cam_out_SOLS,cam_out_SOLL,cam_out_SOLSD,cam_out_SOLLD
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""train_0""",213.806117,222.244454,229.259879,245.743959,258.337482,265.772467,263.978456,254.879872,243.946147,236.70699,230.934476,226.459626,222.014096,216.91297,210.688514,205.181518,201.649949,199.354256,196.582987,198.08732,200.30267,204.5107,209.294262,214.299322,219.180883,223.644692,227.899269,231.883467,235.859155,239.75449,243.663599,247.494148,251.145756,254.734832,258.118795,261.322883,…,6.3101e-7,3.8035e-7,2.6436e-8,-7.3957e-8,-1.0226e-7,-1.5508e-7,1.8131e-8,1.8428e-7,-1.9102e-8,1.8966e-7,-3.4925e-8,-2.5049e-7,-3.5119e-7,3.1814e-8,-0.000001,-0.000002,-7.6918e-7,-0.000007,-0.000007,-0.000021,-0.000008,0.000024,0.00002,0.000008,0.000003,2.1486e-7,-0.000001,-6.1321e-7,0.000001,0.0,349.564325,0.0,2.9477e-12,0.0,0.0,0.0,0.0


In [18]:
df[["state_t_45", "state_q0001_45", "state_q0002_45", "state_q0003_45"]]

state_t_45,state_q0001_45,state_q0002_45,state_q0003_45
f64,f64,f64,f64
279.949099,0.003745,0.000005,3.6643e-8


In [19]:
e_sat_liq = sat_vapor_press_liq(279.949099)
e_sat_ice = sat_vapor_press_ice(279.949099)

In [ ]:
pressure = pre[46]

# 飽和混合比の計算
q_sat_liq = ep * e_sat_liq / (pressure - e_sat_liq)
q_sat_ice = ep * e_sat_ice / (pressure - e_sat_ice)